<a href="https://colab.research.google.com/github/Dexter0013/Face-detection-using-YOLO-model/blob/main/Face_detection_using_YOLO_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/drive/MyDrive/dataYOLO

In [ ]:
!cp -r /content/runs /content/drive/MyDrive/dataYOLO

In [ ]:
!pip install streamlit

In [ ]:
!pip install streamlit_webrtc

In [ ]:
from ultralytics import YOLO

# Load the pretrained YOLO11 Nano model
model = YOLO("/content/drive/MyDrive/data/optimized/runs/yolo11_optimized_run3/weights/best.pt")

# Run inference on an image
results = model("/content/drive/MyDrive/data/dataYOLO/images/20250701_155225.jpg", save=True,
project="/content/PredictedImages", name="auto_saved", exist_ok=True)
# Display results
results[0].show()

In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase
from ultralytics import YOLO
import av
import cv2 # Import cv2 for converting AvFrame to numpy array

# Load YOLO11 Nano model
# Make sure this path is correct and the model exists after training
model = YOLO("/content/drive/MyDrive/dataYOLO/runs/detect/train2/weights/best.pt")

# Define frame processor
class YOLOTransformer(VideoTransformerBase):
    def recv(self, frame: av.VideoFrame) -> av.VideoFrame:
        img = frame.to_ndarray(format="bgr24")

        # Run YOLO inference
        results = model(img)

        # Annotated frame
        annotated_frame = results[0].plot()

        # Convert the annotated frame back to an AvFrame
        return av.VideoFrame.from_ndarray(annotated_frame, format="bgr24")

# Streamlit UI
st.title("YOLO11n + WebRTC Real-Time Object Detection")
st.markdown("This app uses your webcam to detect objects live using YOLO11 Nano.")

# Start WebRTC stream
webrtc_streamer(
    key="yolo11n-stream",
    video_transformer_factory=YOLOTransformer,
    media_stream_constraints={"video": True, "audio": False},
    async_transform=True,
)

In [ ]:
#get the current ip address of the colab notebook
!wget -q -O - ipv4.icanhazip.com

In [ ]:
#Activate local tunnel at the port 8501
!streamlit run app.py & npx localtunnel --port 8501